In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_final_demo = pd.read_csv('../1_data/df_final_demo.txt')
df_final_experiment_clients = pd.read_csv('../1_data/df_final_experiment_clients.txt')
df_final_web_data_pt_1 = pd.read_csv('../1_data/df_final_web_data_pt_1.txt')
df_final_web_data_pt_2 = pd.read_csv('../1_data/df_final_web_data_pt_2.txt')

# Merge web data
df_web = pd.concat([df_final_web_data_pt_1, df_final_web_data_pt_2], ignore_index=True)
df_web["process_step"] = df_web["process_step"].replace('confirm', 'step_4')
df_web["process_step"] = df_web["process_step"].replace('start', 'step_0')
df_web["date_time"] = pd.to_datetime(df_web["date_time"])
df_time = df_web.sort_values(by=['visit_id', 'process_step', 'date_time'])
print(df_time)

#put the name of the steps you wamt to include
steps_required = {'step_0', 'step_1', 'step_2', 'step_3', 'step_4'}
visit_valid = df_time.groupby('visit_id')['process_step'].apply(set).apply(lambda x: steps_required.issubset(x))
df_filtered = df_time[df_time['visit_id'].isin(visit_valid[visit_valid].index)]
print(df_filtered.head())
df_ps_time = pd.DataFrame()
df_ps_time['client_id'] = df_filtered['client_id']
df_ps_time['visit_id'] = df_filtered['visit_id']
df_ps_time['process_step'] = df_filtered['process_step']
df_ps_time['diff_time'] = df_filtered.groupby('visit_id')['date_time'].diff()

#removing all null and negative values
df_ps_time = df_ps_time[df_ps_time['diff_time'] < pd.Timedelta(0)]
print(df_ps_time.head(50))

        client_id             visitor_id                      visit_id  \
108614    3561384   451664975_1722933822  100012776_37918976071_457913   
108613    3561384   451664975_1722933822  100012776_37918976071_457913   
447951    9056452  306992881_89423906595     1000165_4190026492_760066   
447950    9056452  306992881_89423906595     1000165_4190026492_760066   
447949    9056452  306992881_89423906595     1000165_4190026492_760066   
...           ...                    ...                           ...   
539055    7149380  483112224_46340533900  999992932_41666455053_671149   
539054    7149380  483112224_46340533900  999992932_41666455053_671149   
539072    7149380  483112224_46340533900  999992932_41666455053_671149   
539071    7149380  483112224_46340533900  999992932_41666455053_671149   
539070    7149380  483112224_46340533900  999992932_41666455053_671149   

       process_step           date_time  
108614       step_4 2017-04-26 13:22:17  
108613       step_4 2017-04

In [30]:
df_ps_time 

,client_id,visit_id,process_step,diff_time
447950,9056452,1000165_4190026492_760066,step_1,0 days 00:00:03
447949,9056452,1000165_4190026492_760066,step_2,0 days 00:00:24
447948,9056452,1000165_4190026492_760066,step_3,0 days 00:01:17
447947,9056452,1000165_4190026492_760066,step_4,0 days 00:00:37
240557,7338123,100019538_17884295066_43909,step_0,0 days 00:00:49
...,...,...,...,...
539055,7149380,999992932_41666455053_671149,step_1,0 days 00:00:21
539054,7149380,999992932_41666455053_671149,step_2,0 days 00:01:08
539072,7149380,999992932_41666455053_671149,step_3,0 days 00:14:14
539071,7149380,999992932_41666455053_671149,step_4,0 days 00:02:22


PLOT TRENDS

In [38]:
df_merged = df_ps_time.merge(df_final_experiment_clients, on='client_id', how='left')
df_merged.head(200)


,client_id,visit_id,process_step,diff_time,Variation
0,9056452,1000165_4190026492_760066,step_1,0 days 00:00:03,NaN
1,9056452,1000165_4190026492_760066,step_2,0 days 00:00:24,NaN
2,9056452,1000165_4190026492_760066,step_3,0 days 00:01:17,NaN
3,9056452,1000165_4190026492_760066,step_4,0 days 00:00:37,NaN
4,7338123,100019538_17884295066_43909,step_0,0 days 00:00:49,Test
...,...,...,...,...,...
195,663873,100559681_42553631610_873319,step_1,0 days 00:00:31,Test
196,663873,100559681_42553631610_873319,step_2,0 days 00:00:09,Test
197,663873,100559681_42553631610_873319,step_3,0 days 00:01:36,Test
198,663873,100559681_42553631610_873319,step_4,0 days 00:01:07,Test


In [ ]:
#check client that did not participate in the experiment

df_missing_variation = df_merged[df_merged['Variation'].isna()]
print(df_missing_variation.head())  # Show a sample of missing cases
print(f"Total rows missing Variation: {df_missing_variation.shape[0]}")


    client_id                      visit_id process_step       diff_time  \
0     9056452     1000165_4190026492_760066       step_1 0 days 00:00:03   
1     9056452     1000165_4190026492_760066       step_2 0 days 00:00:24   
2     9056452     1000165_4190026492_760066       step_3 0 days 00:01:17   
3     9056452     1000165_4190026492_760066       step_4 0 days 00:00:37   
16    5303781  100034012_10293842356_627828       step_0 0 days 00:01:31   

   Variation  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
16       NaN  
Total rows missing Variation: 217127


In [ ]:
#check number of clients that dont exist in the both DFs

missing_clients = df_ps_time[~df_ps_time['client_id'].isin(df_final_experiment_clients['client_id'])]
print(f"Clients in df_ps_time but missing in df_final_experiment_clients: {missing_clients.shape[0]}")
print(missing_clients.head())


Clients in df_ps_time but missing in df_final_experiment_clients: 154033
        client_id                      visit_id process_step       diff_time
447950    9056452     1000165_4190026492_760066       step_1 0 days 00:00:03
447949    9056452     1000165_4190026492_760066       step_2 0 days 00:00:24
447948    9056452     1000165_4190026492_760066       step_3 0 days 00:01:17
447947    9056452     1000165_4190026492_760066       step_4 0 days 00:00:37
603155    5303781  100034012_10293842356_627828       step_0 0 days 00:01:31


In [43]:
# drop the clients that dont exist in both

df_merged = df_merged.dropna(subset=['Variation'])
print(f"Remaining rows after removing missing Variation: {df_merged.shape[0]}")
print(df_merged['Variation'].unique())  # Should only show ['Test', 'Control']
df_merged.head(100)

Remaining rows after removing missing Variation: 157847
['Test' 'Control']


,client_id,visit_id,process_step,diff_time,Variation
4,7338123,100019538_17884295066_43909,step_0,0 days 00:00:49,Test
5,7338123,100019538_17884295066_43909,step_0,0 days 00:00:14,Test
6,7338123,100019538_17884295066_43909,step_1,0 days 00:00:23,Test
7,7338123,100019538_17884295066_43909,step_1,0 days 00:00:06,Test
8,7338123,100019538_17884295066_43909,step_1,0 days 00:00:23,Test
...,...,...,...,...,...
252,9956593,100733473_61604582110_215085,step_3,0 days 00:01:57,Control
253,9956593,100733473_61604582110_215085,step_4,0 days 00:01:52,Control
254,5392133,100738555_11362326861_913293,step_1,0 days 00:00:12,Test
255,5392133,100738555_11362326861_913293,step_2,0 days 00:00:49,Test
